In [49]:
#!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.3 MB/s eta 0:00:00a 0:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://gi

In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py
#from netCDF4 import Dataset
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pqw

from functools import reduce
import operator
import gc

In [2]:
# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

In [4]:
# PySpark packages
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

#spark = SparkSession.builder \
#    .master("yarn") \
#    .appName("spark-shell") \
#    .config("spark.driver.maxResultSize", "32g") \
#    .config("spark.driver.memory", "32g") \
#    .config("spark.executor.memory", "6g") \
#    .config("spark.executor.cores", "1") \
#    .config("spark.executor.instances", "30") \
#    .getOrCreate()

#    .config("spark.executor.memory", "14g") \
#    .config("spark.executor.cores", "2") \
#    .config("spark.executor.instances", "60") \
#    .getOrCreate()

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("spark://sohnic:7077") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.jars.packages", "graphframes:graphframes:0.7.0-spark2.4-s_2.11") \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir("hdfs://sohnic:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lshin/.ivy2/cache
The jars for the packages stored in: /home/lshin/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4195f8d-d737-4c27-8b38-35f7c4872b8c;1.0
	confs: [default]
	found graphframes#graphframes;0.7.0-spark2.4-s_2.11 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 112ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.7.0-spark2.4-s_2.11 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------

In [5]:
sc.getConf().getAll()[:10]

[('spark.driver.memory', '32g'),
 ('spark.app.initial.file.urls',
  'spark://sohnic:35535/files/org.slf4j_slf4j-api-1.7.16.jar,spark://sohnic:35535/files/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar'),
 ('spark.master', 'spark://sohnic:7077'),
 ('spark.jars',
  'file:///home/lshin/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/lshin/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.app.startTime', '1723205915735'),
 ('spark.app.submitTime', '1723205915576'),
 ('spark.executor.memory', '6g'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.pyFiles',
  '/home/lshin/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,/home/lshin/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.driver.host', 'sohnic')]

In [6]:
#h5dir = '/mnt/raid5/lshin/'
#flist = !ls /mnt/raid5/lshin

h5dir = '/home/lshin/TNG300/snap99/'
flist = !ls /home/lshin/TNG300/snap99

h5list = flist[1:]#[2:]
#print(flist)
#print(len(h5list))
#print(h5list)

['snap099_sorted_x0_y0_z0.csv', 'snap099_sorted_x0_y0_z1.csv', 'snap099_sorted_x0_y0_z2.csv', 'snap099_sorted_x0_y0_z3.csv', 'snap099_sorted_x0_y0_z4.csv', 'snap099_sorted_x0_y0_z5.csv', 'snap099_sorted_x0_y1_z0.csv', 'snap099_sorted_x0_y1_z1.csv', 'snap099_sorted_x0_y1_z2.csv', 'snap099_sorted_x0_y1_z3.csv', 'snap099_sorted_x0_y1_z4.csv', 'snap099_sorted_x0_y1_z5.csv', 'snap099_sorted_x0_y2_z0.csv', 'snap099_sorted_x0_y2_z1.csv', 'snap099_sorted_x0_y2_z2.csv', 'snap099_sorted_x0_y2_z3.csv', 'snap099_sorted_x0_y2_z4.csv', 'snap099_sorted_x0_y2_z5.csv', 'snap099_sorted_x0_y3_z0.csv', 'snap099_sorted_x0_y3_z1.csv', 'snap099_sorted_x0_y3_z2.csv', 'snap099_sorted_x0_y3_z3.csv', 'snap099_sorted_x0_y3_z4.csv', 'snap099_sorted_x0_y3_z5.csv', 'snap099_sorted_x0_y4_z0.csv', 'snap099_sorted_x0_y4_z1.csv', 'snap099_sorted_x0_y4_z2.csv', 'snap099_sorted_x0_y4_z3.csv', 'snap099_sorted_x0_y4_z4.csv', 'snap099_sorted_x0_y4_z5.csv', 'snap099_sorted_x0_y5_z0.csv', 'snap099_sorted_x0_y5_z1.csv', 'snap09

In [7]:
numh5list = len(h5list)
print(numh5list)

600


In [8]:
h5dir+h5list[0]

'/home/lshin/TNG300/snap99/snap099_sorted_x0_y0_z0.csv'

In [9]:
h5py.is_hdf5(h5dir+h5list[0])

False

In [10]:
try:
    h5f = h5py.File(h5dir+h5list[0], "r")
except IOError as e:
    print("Error opening HDF5 file:", str(e))
# Don't forget f.close() when done! 

Error opening HDF5 file: Unable to synchronously open file (file signature not found)


In [11]:
%%time
df = pd.read_csv(h5dir+h5list[0])
df.head()
#h5f.keys()


CPU times: user 588 ms, sys: 54.8 ms, total: 642 ms
Wall time: 642 ms


,px,py,pz,vx,vy,vz,mass
0,15389.089310,954.616907,4401.295627,-119.992165,-28.731781,-266.06906,0.000588
1,15389.030575,954.669960,4401.003768,-101.488754,12.874359,-215.97354,0.000428
2,15389.011885,955.174487,4400.624592,-84.585670,-22.560968,-274.01807,0.000623
3,15389.240237,955.166115,4401.306553,-48.979668,-38.064640,-236.63635,0.000476
4,15389.082537,955.356952,4401.132427,-76.940155,-1.374161,-279.79910,0.000767


In [12]:
#for key in h5f.keys():
#    item = h5f[key]    
#    if isinstance(item, h5py.Group):  # Check if it's a group
#        print(f"Group: {key}")
#    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
#        print(f"Dataset: {key}, dtype: {item.dtype}")
#    else:
#        print(f"Unknown item: {key}")

In [13]:
#h5f['PartType4'].keys()

In [14]:
#for key in h5f['PartType4'].keys():
#    item = h5f['PartType4'][key]
#    
#    if isinstance(item, h5py.Group):  # Check if it's a group
#        print(f"Group: {key}")
#    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
#        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
#    else:
#        print(f"Unknown item: {key}")

# Save selected features as a parquet

In [15]:
#schema = T.StructType([\
##                       T.StructField('filename',T.StringType(), True),\
#                       T.StructField('gas_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('gas_mass',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('dm_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_form_time',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_mass',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('bh_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('bh_mass',T.ArrayType(T.FloatType(),True), True),\
#                   ])
#schema = T.StructType([\
#                       T.StructField('filename',T.StringType(), True),\
#                       T.StructField('star_coord_x',T.FloatType(), True),\
#                       T.StructField('star_coord_y',T.FloatType(), True),\
#                       T.StructField('star_coord_z',T.FloatType(), True),\
#                       T.StructField('gas_mass',T.FloatType(), True),\
#                       T.StructField('dm_coord',T.FloatType(), True),\
#                       T.StructField('star_coord',T.FloatType(), True),\
#                       T.StructField('star_form_time',T.FloatType(), True),\
#                       T.StructField('star_mass',T.FloatType(), True),\
#                       T.StructField('bh_coord',T.FloatType(), True),\
#                       T.StructField('bh_mass',T.FloatType(), True),\
#                      ])
schema = T.StructType([\
#                       T.StructField('filename',T.StringType(), True),\
                       T.StructField('px',T.FloatType(), True),\
                       T.StructField('py',T.FloatType(), True),\
                       T.StructField('pz',T.FloatType(), True),\
                       T.StructField('vx',T.FloatType(), True),\
                       T.StructField('vy',T.FloatType(), True),\
                       T.StructField('vz',T.FloatType(), True),\
                       T.StructField('mass',T.FloatType(), True),\
#                       T.StructField('star_mass',T.FloatType(), True),\
#                       T.StructField('bh_coord',T.FloatType(), True),\
#                       T.StructField('bh_mass',T.FloatType(), True),\
                      ])

In [16]:
#h5f['PartType4/Coordinates'].shape
len(df)

829890

In [19]:
#h5f['PartType4/Coordinates'][:5]

In [20]:
#star_coord_x = h5f['PartType4/Coordinates'][:1000][:,0].tolist()
#star_coord_y = h5f['PartType4/Coordinates'][:1000][:,1].tolist()
#star_coord_z = h5f['PartType4/Coordinates'][:1000][:,2].tolist()
px = df['px'].tolist()
py = df['py'].tolist()
pz = df['pz'].tolist()
vx = df['vx'].tolist()
vy = df['vy'].tolist()
vz = df['vz'].tolist()
mass = df['mass'].tolist()

In [21]:
#gas_coord = h5f['PartType0/Coordinates'][:,:].flatten().tolist()
#gas_mass =  [float(i) for i in h5f['PartType0/Masses']]
#dm_coord = h5f['PartType1/Coordinates'][:,:].flatten().tolist()
#star_coord = h5f['PartType4/Coordinates'][:,:].flatten().tolist()
#star_form_time =  [float(i) for i in h5f['PartType4/GFM_StellarFormationTime']]
#star_mass =  [float(i) for i in h5f['PartType4/Masses']]
#bh_coord = h5f['PartType5/Coordinates'][:,:].flatten().tolist()
#bh_mass = [float(i) for i in h5f['PartType5/Masses']]

In [22]:
%%time
#sparkdf = spark.createDataFrame(zip(star_coord_x, star_coord_y, star_coord_z),schema)
sparkdf = spark.createDataFrame(zip(px, py, pz, vx, vy, vz, mass),schema)
#sparkdf = spark.createDataFrame(zip(gas_coord_x),schema)

CPU times: user 3.21 s, sys: 48.8 ms, total: 3.26 s
Wall time: 4.69 s


In [23]:
%%time
sparkdf.show(3,truncate=True)

+---------+--------+---------+-----------+----------+----------+------------+
|       px|      py|       pz|         vx|        vy|        vz|        mass|
+---------+--------+---------+-----------+----------+----------+------------+
|15389.089|954.6169|4401.2954|-119.992165|-28.731781|-266.06906| 5.879042E-4|
| 15389.03|  954.67| 4401.004|-101.488754| 12.874359|-215.97354|4.2798722E-4|
|15389.012|955.1745|4400.6245|  -84.58567|-22.560968|-274.01807| 6.225057E-4|
+---------+--------+---------+-----------+----------+----------+------------+
only showing top 3 rows

CPU times: user 3.25 ms, sys: 2.56 ms, total: 5.81 ms
Wall time: 1.52 s


In [24]:
sparkdf.count()

829890

In [25]:
sparkdf.printSchema()

root
 |-- px: float (nullable = true)
 |-- py: float (nullable = true)
 |-- pz: float (nullable = true)
 |-- vx: float (nullable = true)
 |-- vy: float (nullable = true)
 |-- vz: float (nullable = true)
 |-- mass: float (nullable = true)



In [26]:
#outdir = 'hdfs://sohnic:54310/user/lshin/'
outdir = 'hdfs://sohnic:54310/data/TNG300/snap99/'

In [27]:
outname = outdir+h5list[0].replace("csv","parquet.snappy")
print(outname)

hdfs://sohnic:54310/data/TNG300/snap99/snap099_sorted_x0_y0_z0.parquet.snappy


In [28]:
%%time
sparkdf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outname)

CPU times: user 13.7 ms, sys: 6.92 ms, total: 20.6 ms
Wall time: 2.76 s


# Check up the parquet

In [45]:
%%time
# Read all parquets in the directory
newsparkdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(outname)

CPU times: user 4.33 ms, sys: 8 µs, total: 4.34 ms
Wall time: 124 ms


In [46]:
newsparkdf.printSchema()

root
 |-- px: float (nullable = true)
 |-- py: float (nullable = true)
 |-- pz: float (nullable = true)
 |-- vx: float (nullable = true)
 |-- vy: float (nullable = true)
 |-- vz: float (nullable = true)
 |-- mass: float (nullable = true)



In [47]:
%%time
print(newsparkdf.count())

1206812
CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 254 ms


In [48]:
%%time
newsparkdf.limit(2).toPandas().transpose()

CPU times: user 8.11 ms, sys: 3.91 ms, total: 12 ms
Wall time: 353 ms


,0,1
px,204866.796875,204869.765625
py,204772.015625,204746.625000
pz,197395.640625,197410.687500
vx,9.041390,254.601059
vy,-134.318375,-92.379158
vz,-66.458954,-7.044174
mass,0.000831,0.000476


In [49]:
newsparkdf.select(['px','py','pz']).show(3)
#sparkdf.select(['star_coord_x']).show(3)

+---------+---------+---------+
|       px|       py|       pz|
+---------+---------+---------+
| 204866.8|204772.02|197395.64|
|204869.77|204746.62|197410.69|
|204835.03|204758.92|197448.14|
+---------+---------+---------+
only showing top 3 rows



In [38]:
%%time
newsparkdf.select(['px','py','pz']).describe().toPandas().set_index('summary').transpose()
#sparkdf.select(['gas_coord_x']).describe().toPandas().set_index('summary').transpose()

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 700 ms


summary,count,mean,stddev,min,max
px,829890,7333.757431926775,6489.1686268262965,0.35823357,20499.875
py,829890,7210.045494197404,4223.622205323317,0.09408737,20467.947
pz,829890,24904.023832161158,9929.996396301181,5.216663,34166.35


# repartition

In [39]:
%%time
newsparkdf.cache()
newsparkdf.repartition(10,"px").count()

CPU times: user 4.39 ms, sys: 0 ns, total: 4.39 ms
Wall time: 756 ms


829890

# Converting all files

In [44]:
%%time
for i in range(len(h5list)):
    df = pd.read_csv(h5dir+h5list[i])
    px = df['px'].tolist()
    py = df['py'].tolist()
    pz = df['pz'].tolist()
    vx = df['vx'].tolist()
    vy = df['vy'].tolist()
    vz = df['vz'].tolist()
    mass = df['mass'].tolist()
    sparkdf = spark.createDataFrame(zip(px, py, pz, vx, vy, vz, mass),schema)
    
    outname = outdir+h5list[i].replace("csv","parquet.snappy")
    sparkdf.write.option("compression", "snappy").mode("overwrite").save(outname)
    print(i)

0


1


2


3


4


...

595


596


597


598


599
CPU times: user 53min 5s, sys: 1min 13s, total: 54min 18s
Wall time: 1h 11min 38s
